#Hi there


---

**Resources**
*   https://www.geeksforgeeks.org/what-is-sentiment-analysis/ (intro)
*   https://www.nltk.org/api/nltk.tokenize.html (tokenize - separate sentence into words)
*   https://youtu.be/9p1KYtYAus8 (vader lexicon tutorial - for sentiment analysis)
*   dataset (#56 from https://www.nltk.org/nltk_data/)
*   dataset downloaded from https://www.kaggle.com/datasets/mdwaquarazam/headphone-dataset-review-analysis

<br>


<br>
Documents (bg study, methods etc will do later)

> Step 1: Count vectorizer (frequency each words appeared), then identify similarity (use cosine similarity equation, correlation etc.)

> Step 2: Tokenize, Pre-process comments, then do sentiment analysis

> Step 3: each of us will choose our preferred method (bayes, knn, k-means)

> Step 4: compare the results (from using different methods)

*lastly, complete our documents then submit :)







In [4]:
%pip install nltk
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# enter: pip install nltk  (when using python)

import nltk
import pandas as pd    # redundant if run the above codes
import random

In [7]:
data = pd.read_csv('jeromechinjs/NLP-Assignment/headphone_datn.csv')
nltk.download()
nltk.download('state_union')
nltk.download('stopwords')

FileNotFoundError: [Errno 2] No such file or directory: 'jeromechinjs/NLP-Assignment/headphone_datn.csv'

In [ ]:
data.head()
total = data.count()
print(total)

In [ ]:
words = [w for w in nltk.corpus.state_union.words() if w.isalpha()] # list out all the individual words
stopwords = nltk.corpus.stopwords.words("english")
words = [w for w in words if w.lower() not in stopwords] # change all the capital letter to small letter

# tbc......////